In [ ]:
import concurrent.futures
import os
import sys
from functools import partial

sys.path.append('..')

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from safe_module import distance

In [ ]:
file_path = "../../../MSAT/examples/lowermost/TX2008.V1.P010.dat"
data = np.loadtxt(file_path)

n_material = np.size(data,0)

lats = data[:,0]
lons = data[:,1]
rhos = data[:,2]
cijs = data[:,3:24]

c_vecs = [cijs[i,:] for i in range(n_material)]

In [ ]:
symmetry_class = ['XISO']
num_workers = os.cpu_count()

betas = np.empty((len(symmetry_class), n_material))
thetas = np.empty((len(symmetry_class), n_material))
sigmas = np.empty((len(symmetry_class), n_material))
phis = np.empty((len(symmetry_class), n_material))

tracker = np.arange(n_material)

for i, sigma in enumerate(symmetry_class):

    print(f'running minimizer for symmetry class: {sigma}')

    distance_new = partial(distance, use_parallel_processing=False)
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = list(executor.map(distance_new, c_vecs, [sigma]*n_material, tracker))

    for j in range(n_material):
        betas[i,j], thetas[i,j], sigmas[i,j], phis[i,j] = futures[j]

In [ ]:
def global_plot(values, title, figname=None,
                reverse_cmap=False, vmin=None, vmax=None, ticks=None, extend='neither'):

    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    ax.set_global()
    ax.coastlines()

    cmap = plt.get_cmap('jet').reversed()

    if reverse_cmap:
        cmap = cmap.reversed()

    if vmin == None and vmax == None:
        sc = ax.scatter(lons, lats, c=values[0,:],
                        cmap=cmap, alpha=0.6, edgecolors='w', linewidth=0.5, transform=ccrs.PlateCarree())
    else:
        sc = ax.scatter(lons, lats, c=values[0,:], vmin=vmin, vmax=vmax,
                        cmap=cmap, alpha=0.6, edgecolors='w', linewidth=0.5, transform=ccrs.PlateCarree())
        
    clb = plt.colorbar(sc, orientation='vertical', extend=extend, fraction=0.0235, pad=0.05)
    if ticks is not None:
        clb.set_ticks(ticks)
    clb.ax.tick_params(labelsize=10)
    ax.set_title(title, fontsize=20)
    
    if figname:
        #plt.savefig(f'{figname}.png')
        pass
    
    plt.show()

In [ ]:
# plot for betas

parameter = betas
title = r'$\beta_{XISO}$ (°)'
filename = 'betas'
global_plot(parameter, title, filename, reverse_cmap=True, vmin=0, vmax=2.0, extend='max')

In [ ]:
# plot for thetas

parameter = abs(np.cos(thetas))
title = '|cos (azimuthal angle)|'
filename = 'azimuthal_angle'
global_plot(parameter, title, filename, vmin=0, vmax=1)

In [ ]:
# plot for betas with points corresponding to sub-vertical phis masked

parameter = abs(np.cos(thetas))
parameter[np.cos(2*phis)>0.5] = np.nan
title = '|cos (azimuthal angle)|'
filename = 'azimuthal_angle_subverticals_masked'
global_plot(parameter, title, filename, vmin=0, vmax=1)

In [ ]:
# plot for sigmas

parameter = sigmas*180/np.pi
title = r'$\sigma$ (°)'
global_plot(parameter, title)

In [ ]:
# plot for phis

parameter = np.cos(2*phis)
title = f'cos (2 * polar angle)'
filename = 'polar_angle'
global_plot(parameter, title, filename, vmin=-1, vmax=1, ticks=np.arange(-1,1.1,0.2))

In [ ]:
# plot for anisotropy index (MSAT)

data = sp.io.loadmat(r'Data/anisotropy_index_MSAT.mat')
parameter = np.array(data['C'])
title = 'universal_elastic_anisotropy_index'
filename = title
global_plot(parameter, title, filename, vmin=0, vmax=0.1, extend='max')

In [ ]:
# plot for proportion of hexagonality (MSAT)

data = sp.io.loadmat(r'Data/hexagonality_MSAT.mat')
parameter = np.array(data['H'])
title = 'proportion_of_hexagonality'
filename = title
global_plot(parameter, title, filename, vmin=0, vmax=1)

In [ ]:
data = sp.io.loadmat(r'Data/hexagonality_MSAT.mat')
x = np.array(data['H']).reshape(2556)
y = betas.reshape(2556)

number_of_bins = 20
bins = np.linspace(0, 1, number_of_bins+1)
bin_width = bins[1] - bins[0]
index = np.digitize(x, bins[:-1]) - 1

bin_mids = bins[:-1] + bin_width / 2

running_median = np.array([np.median(y[index == i]) for i in range(number_of_bins)])
running_mad = np.array([np.median(np.abs(y[index == i] - running_median[i])) for i in range(number_of_bins)])

plt.figure(figsize=(6,6))
plt.scatter(x, y)
plt.plot(bin_mids, running_median, c='r')
plt.plot(bin_mids, running_median + running_mad, 'r--')
plt.plot(bin_mids, running_median - running_mad, 'r--')
plt.xticks(np.linspace(0,1,11))
plt.xlabel('proportion of hexagonality', fontsize=15)
plt.ylabel(r'$\beta_{XISO}$ (°)', fontsize=15)

plt.savefig('beta_xiso_vs_hex.png')
plt.show()